In [1]:
!pip install -q transformers datasets evaluate

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

# Load CIFAR-10 dataset
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [3]:
# Normalize data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert class vectors to binary class matrices
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)

In [4]:
# import model from experiment 3
from keras.models import load_model
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam


imported_model = load_model('trained_tiny_imagenet_model.h5')

imported_model.summary()

# Create a new input layer that matches the size of Tiny ImageNet images
input_shape = (32, 32, 3)
new_input = Input(shape=input_shape)

# Pass new_input through the layers of the old model, except for the output layers
x = new_input
for layer in imported_model.layers[:-3]:  # skip the last three layers (Flatten, Dense, Dense)
    x = layer(x)

# Calculate the correct output dimensions after the last pooling layer
shape_before_flatten = x.shape[1]
print("Shape before flattening:", shape_before_flatten)

x = Dense(1024, activation='relu', input_shape=(shape_before_flatten,), kernel_regularizer=l2(0.01))(x)  # Adjusted dense layer
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)  # New output layer for 10 classes

# Learning rate scheduler function
def scheduler(epoch, lr):
    if epoch < 55:
        return lr
    else:
        return lr * np.exp(-0.1)

# Create new model
new_model = Model(inputs=new_input, outputs=x)
new_model.summary()






Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 64)        1792      
                                                                 
 batch_normalization_4 (Bat  (None, 64, 64, 64)        256       
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 64)        36928     
                                                                 
 batch_normalization_5 (Bat  (None, 64, 64, 64)        256       
 chNormalization)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 32, 32, 64)        0     

In [5]:
from keras.preprocessing.image import ImageDataGenerator
# data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest',
    zoom_range=0.1
)
datagen.fit(X_train)

checkpoint = ModelCheckpoint('exp4_model.keras', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', verbose=1, patience=10, restore_best_weights=True)

In [7]:
lr_schedule = LearningRateScheduler(scheduler)

# Compile the new model
new_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = new_model.fit(datagen.flow(X_train, Y_train, batch_size=64),
                     epochs=200,
                     validation_data=(X_test, Y_test),
                     callbacks=[early_stopping, checkpoint, lr_schedule])

Epoch 1/200
782/782 [==============================] - 30s 35ms/step - loss: 1.3488 - accuracy: 0.6443 - val_loss: 1.3996 - val_accuracy: 0.6122 - lr: 1.0000e-04
Epoch 2/200
782/782 [==============================] - 27s 35ms/step - loss: 1.2111 - accuracy: 0.6570 - val_loss: 1.2816 - val_accuracy: 0.6226 - lr: 1.0000e-04
Epoch 3/200
782/782 [==============================] - 27s 35ms/step - loss: 1.1399 - accuracy: 0.6639 - val_loss: 1.1781 - val_accuracy: 0.6459 - lr: 1.0000e-04
Epoch 4/200
782/782 [==============================] - 27s 34ms/step - loss: 1.1060 - accuracy: 0.6706 - val_loss: 1.1970 - val_accuracy: 0.6377 - lr: 1.0000e-04
Epoch 5/200
782/782 [==============================] - 27s 34ms/step - loss: 1.0913 - accuracy: 0.6687 - val_loss: 1.2201 - val_accuracy: 0.6255 - lr: 1.0000e-04
Epoch 6/200
782/782 [==============================] - 27s 34ms/step - loss: 1.0853 - accuracy: 0.6673 - val_loss: 1.2311 - val_accuracy: 0.6260 - lr: 1.0000e-04
Epoch 7/200
782/782 [=======

In [9]:
model_scores = new_model.evaluate(X_test, Y_test, verbose=1)
print(f'Model Test accuracy: {model_scores[1]*100}%')

313/313 [==============================] - 1s 3ms/step - loss: 1.0616 - accuracy: 0.6708
Model Test accuracy: 67.0799970626831%


In [10]:
import pandas as pd

pd.DataFrame(history.history).to_csv('exp4_history.csv')